In [1]:
import pandas as pd
import os
from env import get_db_url
import matplotlib as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import wrangle_zillow

### Acquire

In [2]:
df = wrangle_zillow.get_zillow_data()

In [3]:
df.head()

,heatingorsystemtypeid,parcelid,storytypeid,typeconstructiontypeid,airconditioningtypeid,propertylandusetypeid,architecturalstyletypeid,id,basementsqft,bathroomcnt,...,max_transactiondate,id,logerror,transactiondate,architecturalstyledesc,propertylandusedesc,airconditioningdesc,typeconstructiondesc,storydesc,heatingorsystemdesc
0,NaN,14297519,NaN,NaN,NaN,261.0,NaN,1727539,NaN,3.5,...,2017-01-01,0,0.025595,2017-01-01,None,Single Family Residential,None,None,None,None
1,NaN,17052889,NaN,NaN,NaN,261.0,NaN,1387261,NaN,1.0,...,2017-01-01,1,0.055619,2017-01-01,None,Single Family Residential,None,None,None,None
2,NaN,14186244,NaN,NaN,NaN,261.0,NaN,11677,NaN,2.0,...,2017-01-01,2,0.005383,2017-01-01,None,Single Family Residential,None,None,None,None
3,2.0,12177905,NaN,NaN,NaN,261.0,NaN,2288172,NaN,3.0,...,2017-01-01,3,-0.103410,2017-01-01,None,Single Family Residential,None,None,None,Central
4,2.0,12095076,NaN,NaN,1.0,261.0,NaN,781532,NaN,3.0,...,2017-01-01,6,-0.001011,2017-01-01,None,Single Family Residential,Central,None,None,Central


### Prepare

In [4]:
# remove rows and columns with more than 30% missing data
df = wrangle_zillow.handle_missing_values(df, .7,.7)

In [5]:
# view remaining null values
df.isna().mean()

parcelid                        0.000000
propertylandusetypeid           0.000000
id                              0.000000
bathroomcnt                     0.000000
bedroomcnt                      0.000000
calculatedbathnbr               0.002574
calculatedfinishedsquarefeet    0.001526
finishedsquarefeet12            0.004672
fips                            0.000000
fullbathcnt                     0.002574
latitude                        0.000000
longitude                       0.000000
lotsizesquarefeet               0.006999
propertycountylandusecode       0.000000
rawcensustractandblock          0.000000
regionidcity                    0.019737
regionidcounty                  0.000000
regionidzip                     0.000458
roomcnt                         0.000000
yearbuilt                       0.002174
structuretaxvaluedollarcnt      0.001564
taxvaluedollarcnt               0.000019
assessmentyear                  0.000000
landtaxvaluedollarcnt           0.000019
taxamount       

In [6]:
# drop remaining nulls
df = df.dropna()

In [7]:
df.shape

(50673, 31)

In [8]:
# split data
train, validate, test = wrangle_zillow.split_zillow_data(df)

In [10]:
train.shape, validate.shape, test.shape

((28376, 31), (12162, 31), (10135, 31))